In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import CRUD Python module file name and class name
from aac_lib import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Connect to database via CRUD Module
db = AnimalShelter()

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# drop column['_id'] to avoid invalid object crash
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.info())
#print(df.head())


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Add in Grazioso Salvare’s logo
image_filename = '/home/seantoon_snhu/Desktop/Grazioso Salvare Logo2.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#Unique Identifier and attach www.snhu.edu to photo
app.layout = html.Div(
    style={'backgroundColor': '#FFE1E6'},  # Set background color here
    children=[
        html.A([
            html.Center(html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                height=250, width=251))], href='https://www.snhu.edu', target="_blank"),
        html.Center(html.B(html.H1('Austin Animal Center - Sean Toon'))),
        html.Hr(),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue Type', 'value': 'Water Rescue Type'},
                {'label': 'Mountain/ Wilderness Rescue Type', 'value': 'Mountain/ Wilderness Rescue Type'},
                {'label': 'Disaster/Individual Rescue Type', 'value': 'Disaster/Individual Rescue Type'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset'
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        #Set up the features for interactive data table
        page_size=10, #set number of rows per page
        page_action="native",
        sort_action="native", #enable sorting
        style_table={'overflowX': 'scroll'}, #add scroll to list
        row_selectable='single', #single-row selection
        selected_rows = [0]
    
    ),
    html.Br(),
    html.Hr(),
    
    #Set up pie chart and geolocation chart side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Filter interactions
    if filter_type == 'Water Rescue Type':
        df = pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))
            
    elif filter_type == 'Mountain/ Wilderness Rescue Type':
        df = pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                                "Siberian Husky", "Rottweiler"
                                 ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))

    elif filter_type == 'Disaster/Individual Rescue Type':
        df = pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                                "Bloodhound","Rottweiler"
                                 ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
        }))
            
    # Filter reset 
    else:
        df = pd.DataFrame.from_records(db.read({}))
    df.drop(columns=['_id'],inplace=True)
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        #Debug
        #print(df.columns)
        #print(df['breed'].unique())
        #print(df[df['breed'] == 'Chesapeake Bay Retriever'])

    return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    dff= pd.DataFrame.from_dict(viewData)
    return [
        
        dcc.Graph(            
            figure = px.pie(dff, names='breed', title='Preferred Animals')
        )   
    ]

    
#Callback to highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
#geolocation chart
   
    dff = pd.DataFrame.from_dict(viewData)
    
    if dff.empty: #check to see if dff is empty
        return []
    
    if index is None:
        row = 0
    else:
        row = index[0]
        
    return[
        dl.Map(style={'width': '1000px', 'height': '500px'},
              center=[30.75, -97.48], zoom=10, children=[
                   dl.TileLayer(id="base_layer_id"),
                   dl.Marker(position=[dff.iloc[row,13], dff.iloc[row,14]],
                             children=[
                                 dl.Tooltip(dff.iloc[row,4]),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(dff.iloc[row,9])
                                 ])
                             ])
               ])
    ]
app.run_server(debug=True)

Dash app running on http://127.0.0.1:10858/
